In [1]:
import csv
import numpy as np
import Loader
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError

In [70]:
compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

#print(labelsTrain)
#print(compoundsTrain)
#print(smilesTrain)
#print(activitiesTrain)

#for i in range(len(labelsTrain)):
#    print(labelsTrain[i] + ": ", compoundDataTrain[0,i])

def toClassification(y): # The resulting array will contain values of -1 if it is below 4.5 and 1 if it is above
    y = np.array(y)
    classification = (y.astype(float)>4).astype(int)
    return classification * 2 - 1

def normalizeData(train,test,validate):
    for i in range(np.shape(train)[1]):
        std = np.std(train[:,i])
        mean = np.mean(train[:,i])
        if(std == 0):
            std = 1
        train[:,i] = (train[:,i] - mean) / std
        test[:,i] = (test[:,i] - mean) / std
        validate[:,i] = (validate[:,i] - mean) / std
    return train, test, validate

def makeAverage(arr): # Averages the ten values for each score and creates a new array
    arr = np.array(arr)
    newArr = np.empty((np.shape(arr)[0], np.shape(arr)[1] - 18))
    newArr[:,0] = np.mean(arr[:,:10], axis = 1)
    newArr[:,1] = np.mean(arr[:,10:20], axis = 1)
    newArr[:,2:] = arr[:,20:]
    return newArr

def averageScores(train, test, validate): #wrapper function to handle all conversions at once
    newTrain = makeAverage(train)
    newTest = makeAverage(test)
    newValid = makeAverage(validate)
    return newTrain, newTest, newValid
    
compoundDataTrain, compoundDataTest, compoundDataValidate = averageScores(compoundDataTrain, compoundDataTest, compoundDataValidate)
compoundDataTrain, compoundDataTest, compoundDataValidate = normalizeData(compoundDataTrain, compoundDataTest, compoundDataValidate)
print(len(compoundsTrain), len(compoundsTest), len(compoundsValidate))

2717 340 340


In [67]:
print(compoundDataTrain[:,-85:])


[[0. 0. 0. ... 0. 0. 0.]
 [0. 7. 6. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [46]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=20000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)


model = Sequential()
model.add(Dense(200, input_dim=np.shape(compoundDataTrain)[1], activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(150, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(100, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(1, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))

model.compile(loss='MeanSquaredError', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(compoundDataTrain, toClassification(activitiesTrain),validation_data = 
                    (compoundDataTest, toClassification(activitiesTest)),epochs=400, batch_size=32)

Epoch 1/400
85/85 [==============================] - 2s 10ms/step - loss: 5.5820 - accuracy: 0.0000e+00 - val_loss: 5.5588 - val_accuracy: 0.0000e+00
Epoch 2/400
85/85 [==============================] - 0s 6ms/step - loss: 5.5235 - accuracy: 0.0000e+00 - val_loss: 5.5068 - val_accuracy: 0.0000e+00
Epoch 3/400
85/85 [==============================] - 1s 11ms/step - loss: 5.4716 - accuracy: 0.0000e+00 - val_loss: 5.4543 - val_accuracy: 0.0000e+00
Epoch 4/400
85/85 [==============================] - 1s 7ms/step - loss: 5.4207 - accuracy: 0.0000e+00 - val_loss: 5.4034 - val_accuracy: 0.0000e+00
Epoch 5/400
85/85 [==============================] - 1s 11ms/step - loss: 5.3693 - accuracy: 0.0000e+00 - val_loss: 5.3509 - val_accuracy: 0.0000e+00
Epoch 6/400
85/85 [==============================] - 1s 6ms/step - loss: 5.3166 - accuracy: 0.0000e+00 - val_loss: 5.3126 - val_accuracy: 0.0000e+00
Epoch 7/400
85/85 [==============================] - 1s 12ms/step - loss: 5.2661 - accuracy: 0.0000e+00

Epoch 58/400
85/85 [==============================] - 1s 9ms/step - loss: 2.5071 - accuracy: 0.5620 - val_loss: 2.5835 - val_accuracy: 0.5559
Epoch 59/400
85/85 [==============================] - 0s 5ms/step - loss: 2.4734 - accuracy: 0.5631 - val_loss: 2.5545 - val_accuracy: 0.5588
Epoch 60/400
85/85 [==============================] - 0s 5ms/step - loss: 2.4335 - accuracy: 0.5664 - val_loss: 2.5125 - val_accuracy: 0.5412
Epoch 61/400
85/85 [==============================] - 0s 5ms/step - loss: 2.4009 - accuracy: 0.5690 - val_loss: 2.5017 - val_accuracy: 0.5029
Epoch 62/400
85/85 [==============================] - 1s 9ms/step - loss: 2.3707 - accuracy: 0.5679 - val_loss: 2.4497 - val_accuracy: 0.5353
Epoch 63/400
85/85 [==============================] - 0s 5ms/step - loss: 2.3334 - accuracy: 0.5694 - val_loss: 2.4219 - val_accuracy: 0.5618
Epoch 64/400
85/85 [==============================] - 1s 9ms/step - loss: 2.2999 - accuracy: 0.5723 - val_loss: 2.4434 - val_accuracy: 0.5794
Epoch 

85/85 [==============================] - 0s 5ms/step - loss: 0.4369 - accuracy: 0.6117 - val_loss: 0.6163 - val_accuracy: 0.5765
Epoch 173/400
85/85 [==============================] - 1s 11ms/step - loss: 0.4317 - accuracy: 0.6128 - val_loss: 0.6058 - val_accuracy: 0.5765
Epoch 174/400
85/85 [==============================] - 1s 10ms/step - loss: 0.4266 - accuracy: 0.6121 - val_loss: 0.6039 - val_accuracy: 0.5765
Epoch 175/400
85/85 [==============================] - 0s 5ms/step - loss: 0.4211 - accuracy: 0.6124 - val_loss: 0.6051 - val_accuracy: 0.5794
Epoch 176/400
85/85 [==============================] - 0s 5ms/step - loss: 0.4169 - accuracy: 0.6124 - val_loss: 0.5994 - val_accuracy: 0.5794
Epoch 177/400
85/85 [==============================] - 0s 5ms/step - loss: 0.4122 - accuracy: 0.6124 - val_loss: 0.5943 - val_accuracy: 0.5794
Epoch 178/400
85/85 [==============================] - 1s 8ms/step - loss: 0.4073 - accuracy: 0.6124 - val_loss: 0.5806 - val_accuracy: 0.5765
Epoch 179/4

85/85 [==============================] - 0s 5ms/step - loss: 0.2201 - accuracy: 0.6158 - val_loss: 0.4143 - val_accuracy: 0.5735
Epoch 287/400
85/85 [==============================] - 0s 5ms/step - loss: 0.2194 - accuracy: 0.6158 - val_loss: 0.4137 - val_accuracy: 0.5735
Epoch 288/400
85/85 [==============================] - 0s 6ms/step - loss: 0.2187 - accuracy: 0.6158 - val_loss: 0.4130 - val_accuracy: 0.5735
Epoch 289/400
85/85 [==============================] - 0s 6ms/step - loss: 0.2180 - accuracy: 0.6158 - val_loss: 0.4120 - val_accuracy: 0.5735
Epoch 290/400
85/85 [==============================] - 0s 5ms/step - loss: 0.2173 - accuracy: 0.6158 - val_loss: 0.4128 - val_accuracy: 0.5765
Epoch 291/400
85/85 [==============================] - 0s 5ms/step - loss: 0.2167 - accuracy: 0.6158 - val_loss: 0.4098 - val_accuracy: 0.5735
Epoch 292/400
85/85 [==============================] - 1s 11ms/step - loss: 0.2160 - accuracy: 0.6158 - val_loss: 0.4088 - val_accuracy: 0.5735
Epoch 293/40

85/85 [==============================] - 0s 5ms/step - loss: 0.1644 - accuracy: 0.6165 - val_loss: 0.3641 - val_accuracy: 0.5735


In [78]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)


model = Sequential()
model.add(Dense(200, input_dim=np.shape(compoundDataTrain[:,:-85])[1], activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(150, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(100, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))
model.add(Dense(1, activation='tanh', kernel_regularizer = keras.regularizers.L1(0.001)))

model.compile(loss='MeanSquaredError', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(compoundDataTrain[:,:-85], toClassification(activitiesTrain),validation_data = 
                    (compoundDataTest[:,:-85], toClassification(activitiesTest)),epochs=400, batch_size=32)

Epoch 1/400
85/85 [==============================] - 2s 9ms/step - loss: 5.5354 - accuracy: 0.0000e+00 - val_loss: 5.5127 - val_accuracy: 0.0000e+00
Epoch 2/400
85/85 [==============================] - 0s 5ms/step - loss: 5.4780 - accuracy: 0.0000e+00 - val_loss: 5.4616 - val_accuracy: 0.0000e+00
Epoch 3/400
85/85 [==============================] - 0s 5ms/step - loss: 5.4264 - accuracy: 0.0000e+00 - val_loss: 5.4154 - val_accuracy: 0.0000e+00
Epoch 4/400
85/85 [==============================] - 0s 5ms/step - loss: 5.3759 - accuracy: 0.0000e+00 - val_loss: 5.3613 - val_accuracy: 0.0000e+00
Epoch 5/400
85/85 [==============================] - 0s 5ms/step - loss: 5.3277 - accuracy: 0.0000e+00 - val_loss: 5.3112 - val_accuracy: 0.0000e+00
Epoch 6/400
85/85 [==============================] - 1s 10ms/step - loss: 5.2787 - accuracy: 0.0000e+00 - val_loss: 5.2618 - val_accuracy: 0.0000e+00
Epoch 7/400
85/85 [==============================] - 0s 5ms/step - loss: 5.2286 - accuracy: 0.0000e+00 - 

Epoch 115/400
85/85 [==============================] - 1s 7ms/step - loss: 1.1291 - accuracy: 0.5683 - val_loss: 1.2502 - val_accuracy: 0.5412
Epoch 116/400
85/85 [==============================] - 0s 5ms/step - loss: 1.1118 - accuracy: 0.5705 - val_loss: 1.2282 - val_accuracy: 0.5353
Epoch 117/400
85/85 [==============================] - 0s 5ms/step - loss: 1.0933 - accuracy: 0.5705 - val_loss: 1.2144 - val_accuracy: 0.5353
Epoch 118/400
85/85 [==============================] - 0s 5ms/step - loss: 1.0779 - accuracy: 0.5709 - val_loss: 1.1989 - val_accuracy: 0.5412
Epoch 119/400
85/85 [==============================] - 1s 8ms/step - loss: 1.0604 - accuracy: 0.5705 - val_loss: 1.1892 - val_accuracy: 0.5412
Epoch 120/400
85/85 [==============================] - 1s 8ms/step - loss: 1.0463 - accuracy: 0.5701 - val_loss: 1.1712 - val_accuracy: 0.5353
Epoch 121/400
85/85 [==============================] - 1s 8ms/step - loss: 1.0300 - accuracy: 0.5705 - val_loss: 1.1520 - val_accuracy: 0.5382

85/85 [==============================] - 0s 5ms/step - loss: 0.3151 - accuracy: 0.5981 - val_loss: 0.5131 - val_accuracy: 0.5529
Epoch 230/400
85/85 [==============================] - 0s 5ms/step - loss: 0.3125 - accuracy: 0.5981 - val_loss: 0.5060 - val_accuracy: 0.5529
Epoch 231/400
85/85 [==============================] - 0s 5ms/step - loss: 0.3122 - accuracy: 0.5955 - val_loss: 0.5178 - val_accuracy: 0.5618
Epoch 232/400
85/85 [==============================] - 0s 5ms/step - loss: 0.3103 - accuracy: 0.5966 - val_loss: 0.5126 - val_accuracy: 0.5588
Epoch 233/400
85/85 [==============================] - 0s 5ms/step - loss: 0.3116 - accuracy: 0.5959 - val_loss: 0.5063 - val_accuracy: 0.5529
Epoch 234/400
85/85 [==============================] - 1s 7ms/step - loss: 0.3070 - accuracy: 0.5988 - val_loss: 0.5132 - val_accuracy: 0.5559
Epoch 235/400
85/85 [==============================] - 0s 5ms/step - loss: 0.3070 - accuracy: 0.5970 - val_loss: 0.5186 - val_accuracy: 0.5588
Epoch 236/400

85/85 [==============================] - 0s 5ms/step - loss: 0.2274 - accuracy: 0.6066 - val_loss: 0.4711 - val_accuracy: 0.5588
Epoch 344/400
85/85 [==============================] - 0s 5ms/step - loss: 0.2268 - accuracy: 0.6062 - val_loss: 0.4519 - val_accuracy: 0.5529
Epoch 345/400
85/85 [==============================] - 1s 7ms/step - loss: 0.2275 - accuracy: 0.6054 - val_loss: 0.4471 - val_accuracy: 0.5588
Epoch 346/400
85/85 [==============================] - 0s 5ms/step - loss: 0.2264 - accuracy: 0.6051 - val_loss: 0.4669 - val_accuracy: 0.5588
Epoch 347/400
85/85 [==============================] - 1s 7ms/step - loss: 0.2259 - accuracy: 0.6069 - val_loss: 0.4693 - val_accuracy: 0.5588
Epoch 348/400
85/85 [==============================] - 0s 5ms/step - loss: 0.2249 - accuracy: 0.6077 - val_loss: 0.4625 - val_accuracy: 0.5588
Epoch 349/400
85/85 [==============================] - 0s 5ms/step - loss: 0.2243 - accuracy: 0.6069 - val_loss: 0.4676 - val_accuracy: 0.5588
Epoch 350/400

In [ ]:
model = Sequential()
model.add(Dense(50, input_dim=np.shape(compoundDataTrain)[1], activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(50, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(1, activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))

model.compile(loss='MeanSquaredError', optimizer='adam', metrics=['accuracy'])

history = model.fit(compoundDataTrain, toClassification(activitiesTrain),validation_data = (compoundDataTest, toClassification(activitiesTest)), epochs=100, batch_size=32)

In [ ]:
csvfile=open("../Data/trainDataDescriptors.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
trainDescriptors=[]
for lv in rd:
    trainDescriptors.insert(1,lv)
headersDescriptors = trainDescriptors[0]
trainDescriptors = trainDescriptors[1:]

csvfile=open("../Data/trainDockingData.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
trainDocking=[]
for lv in rd:
    trainDocking.insert(1,lv)
headersDocking = trainDocking[0]
trainDocking = trainDocking[1:]


csvfile=open("../Data/testDataDescriptors.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
testDescriptors=[]
for lv in rd:
    testDescriptors.insert(1,lv)
testDescriptors = testDescriptors[1:]

csvfile=open("../Data/testDockingData.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
testDocking=[]
for lv in rd:
    testDocking.insert(1,lv)
testDocking = testDocking[1:]


testing = False
if not testing:
    # Removing the compound id from the middle of the docking data
    trainDocking = np.delete(trainDocking, headersDocking.index("compound_id"), axis=1)
    testDocking = np.delete(testDocking, headersDocking.index("compound_id"), axis=1)

    headersDocking.remove("compound_id")

print(np.shape(trainDocking),np.shape(trainDescriptors),np.shape(testDocking), np.shape(testDescriptors))

trainActivity = []
testActivity = []



# Verifying They Are in the Same Order

In [ ]:
#Only works if compound_id is not deleted above
if testing:
    sameTrain = True
    for i in range(len(trainDocking)):
        sameTrain = sameTrain and (trainDocking[i][headersDocking.index("compound_id")] == trainDescriptors[i][headersDescriptors.index("compound_id")])
        print(trainDocking[i][headersDocking.index("compound_id")],trainDescriptors[i][headersDescriptors.index("compound_id")])
    print(sameTrain)

    sameTest = True
    for i in range(len(testDocking)):
        sameTest = sameTest and (testDocking[i][headersDocking.index("compound_id")] == testDescriptors[i][headersDescriptors.index("compound_id")])
        #print(trainDocking[i][headersDocking.index("compound_id")] == trainDescriptors[i][headersDescriptors.index("compound_id")])
    print(sameTest)

# Combining the Two Datasets

In [ ]:
# Because they are one to one, we can easily combine the datasets together immediately


#print(len(trainDocking[0]))
#Taking out the activity score, as it will become the desired output of the model
if len(trainDocking[0]) == 22:
    trainActivity = np.array(trainDocking)[:,headersDocking.index("activity")]
    testActivity = np.array(testDocking)[:,headersDocking.index("activity")]

    trainDocking = np.delete(trainDocking, headersDocking.index("activity"), axis=1)
    testDocking = np.delete(testDocking, headersDocking.index("activity"), axis=1)
    
    headersDocking.remove("activity")

trainX = np.append(trainDescriptors, trainDocking,1)
trainY = trainActivity

testX = np.append(testDescriptors, testDocking,1)
testY = testActivity
#Appending for some reason reverses both arrays, so the index is actually the row 2717 - ind in excel (idk why)
print(trainY[300], trainX[300])

In [ ]:
trainY = trainActivity
trainY = toClassification(trainY)
print(trainActivity[trainY == 1]) #Gives values of > 4.5